In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
from PIL import Image
import os

base_dir = "medical_dataset"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

classes = ["DISEASE", "HEALTHY"]

for folder in [train_dir, val_dir, test_dir]:
    for cls in classes:
        os.makedirs(os.path.join(folder, cls), exist_ok=True)

def generate_images(path, num):
    for i in range(num):
        img = np.random.randint(0, 256, (128,128,3), dtype=np.uint8)
        Image.fromarray(img).save(os.path.join(path, f"img_{i}.jpg"))

for cls in classes:
    generate_images(os.path.join(train_dir, cls), 120)
    generate_images(os.path.join(val_dir, cls), 40)
    generate_images(os.path.join(test_dir, cls), 40)

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(128,128),
    batch_size=16,
    class_mode='binary'
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=(128,128),
    batch_size=16,
    class_mode='binary'
)

test_data = val_gen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data,
    callbacks=[early_stop]
)

loss, acc = model.evaluate(test_data)
print("Test Accuracy:", acc * 100, "%")

Found 240 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 12s 639ms/step - accuracy: 0.4493 - loss: 0.8963 - val_accuracy: 0.5000 - val_loss: 0.7169
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 609ms/step - accuracy: 0.4930 - loss: 0.6987 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 734ms/step - accuracy: 0.4329 - loss: 0.6932 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 621ms/step - accuracy: 0.5617 - loss: 0.6930 - val_accuracy: 0.5000 - val_loss: 0.6931
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 556ms/step - accuracy: 0.4624 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6931
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.7431 - loss: 0.6926
Test Accuracy: 50.0 %
